In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
pd.set_option('display.max_columns', None)  # display all columns
pd.set_option('display.max_rows', None)  # display all rows

# Import the .csv documents

In [3]:
actor_ori = pd.read_csv("/Users/david/Desktop/IronHack/Projects/SQL_project/data/raw/actor.csv")
category_ori = pd.read_csv("/Users/david/Desktop/IronHack/Projects/SQL_project/data/raw/category.csv")
film_ori = pd.read_csv("/Users/david/Desktop/IronHack/Projects/SQL_project/data/raw/film.csv")
inventory_ori = pd.read_csv("/Users/david/Desktop/IronHack/Projects/SQL_project/data/raw/inventory.csv")
language_ori = pd.read_csv ("/Users/david/Desktop/IronHack/Projects/SQL_project/data/raw/language.csv")
old_HDD_ori = pd.read_csv ("/Users/david/Desktop/IronHack/Projects/SQL_project/data/raw/old_HDD.csv")
rental_ori = pd.read_csv ("/Users/david/Desktop/IronHack/Projects/SQL_project/data/raw/rental.csv")

In [4]:
# Make a copy of the files as a backup

actor = actor_ori.copy()
category = category_ori.copy()
film = film_ori.copy()
inventory = inventory_ori.copy()
language = language_ori.copy()
old_HDD = old_HDD_ori.copy()
rental = rental_ori.copy()

# Data exploration of each column

In [5]:
print(actor.shape)
print(category.shape)
print(film.shape)
print(inventory.shape)
print(language.shape)
print(old_HDD.shape)
print(rental.shape)

(200, 4)
(16, 3)
(1000, 13)
(1000, 4)
(6, 3)
(1000, 5)
(1000, 7)


### actor

In [6]:
actor.head()

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


In [7]:
# Crete a new column with the complete name of the actor/actress in lowercase.

actor = actor.assign(actor_actress_name = (actor['first_name'] + ' ' + actor['last_name']).str.lower())
actor.head()

,actor_id,first_name,last_name,last_update,actor_actress_name
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33,penelope guiness
1,2,NICK,WAHLBERG,2006-02-15 04:34:33,nick wahlberg
2,3,ED,CHASE,2006-02-15 04:34:33,ed chase
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33,jennifer davis
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33,johnny lollobrigida


In [8]:
# Check for duplicated rows based on the actor/actress names.

actor.duplicated(subset=['actor_actress_name']).value_counts()

False    199
True       1
dtype: int64

In [9]:
# Identify and display the duplicated rows.

duplicates = actor.duplicated(subset=['actor_actress_name'], keep=False)
duplicated_rows = actor[duplicates]
duplicated_rows

,actor_id,first_name,last_name,last_update,actor_actress_name
100,101,SUSAN,DAVIS,2006-02-15 04:34:33,susan davis
109,110,SUSAN,DAVIS,2006-02-15 04:34:33,susan davis


In [10]:
# Delete the duplicated row.

actor = actor.drop(index=[109])

In [11]:
# Delete columns: first_name, last_name, last_update.

actor = actor.drop(columns=['first_name', 'last_name', 'last_update'])
actor.head()

,actor_id,actor_actress_name
0,1,penelope guiness
1,2,nick wahlberg
2,3,ed chase
3,4,jennifer davis
4,5,johnny lollobrigida


### category

In [12]:
category.head()

,category_id,name,last_update
0,1,Action,2006-02-15 04:46:27
1,2,Animation,2006-02-15 04:46:27
2,3,Children,2006-02-15 04:46:27
3,4,Classics,2006-02-15 04:46:27
4,5,Comedy,2006-02-15 04:46:27


In [13]:
# Delete the column last_update since it does not contain any useful information.

category = category.drop(columns=['last_update'])
category.head()

,category_id,name
0,1,Action
1,2,Animation
2,3,Children
3,4,Classics
4,5,Comedy


### film

In [14]:
film.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,NaN,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,NaN,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,NaN,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,NaN,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,NaN,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [15]:
film["title"].value_counts().sum()

1000

In [16]:
# Add two columns "subtitles_id" and "original_language_id", and fill them with default values.

film['subtitles_id'] = 0
film["original_language_id"] = 0
film.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update,subtitles_id
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,0,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,0
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,0,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42,0
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,0,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42,0
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,0,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42,0
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,0,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42,0


In [17]:
# Delete columns "last_update" and "rental_duration" as they do not contain useful information.

film = film.drop(columns = "last_update")
film = film.drop(columns = "rental_duration")

film.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_rate,length,replacement_cost,rating,special_features,subtitles_id
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,0,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",0
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,0,4.99,48,12.99,G,"Trailers,Deleted Scenes",0
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,0,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",0
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,0,2.99,117,26.99,G,"Commentaries,Behind the Scenes",0
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,0,2.99,130,22.99,G,Deleted Scenes,0


In [18]:
# Change the name of the column "rental_rate" by "rental_price"

film.rename(columns={"rental_rate": "rental_price"}, inplace=True)

In [19]:
# Fill the columns "rental_price" with default values.

film["rental_price"] = 0
film["replacement_cost"] = 0

In [20]:
film.tail()

,film_id,title,description,release_year,language_id,original_language_id,rental_price,length,replacement_cost,rating,special_features,subtitles_id
995,996,YOUNG LANGUAGE,A Unbelieveable Yarn of a Boat And a Database ...,2006,1,0,0,183,0,G,"Trailers,Behind the Scenes",0
996,997,YOUTH KICK,A Touching Drama of a Teacher And a Cat who mu...,2006,1,0,0,179,0,NC-17,"Trailers,Behind the Scenes",0
997,998,ZHIVAGO CORE,A Fateful Yarn of a Composer And a Man who mus...,2006,1,0,0,105,0,NC-17,Deleted Scenes,0
998,999,ZOOLANDER FICTION,A Fateful Reflection of a Waitress And a Boat ...,2006,1,0,0,101,0,R,"Trailers,Deleted Scenes",0
999,1000,ZORRO ARK,A Intrepid Panorama of a Mad Scientist And a B...,2006,1,0,0,50,0,NC-17,"Trailers,Commentaries,Behind the Scenes",0


In [21]:
film.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_price,length,replacement_cost,rating,special_features,subtitles_id
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,0,0,86,0,PG,"Deleted Scenes,Behind the Scenes",0
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,0,0,48,0,G,"Trailers,Deleted Scenes",0
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,0,0,50,0,NC-17,"Trailers,Deleted Scenes",0
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,0,0,117,0,G,"Commentaries,Behind the Scenes",0
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,0,0,130,0,G,Deleted Scenes,0


### inventory

In [22]:
inventory.head()

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17
2,3,1,1,2006-02-15 05:09:17
3,4,1,1,2006-02-15 05:09:17
4,5,1,2,2006-02-15 05:09:17


In [23]:
# Estimate the number of total films, since the inventory seems incomplete. The maximum value in "inventory_id"
# is much lower than the maximum value in the column "inventory_id" in the table "rental".

x = len(rental['inventory_id'].unique())
y = inventory["film_id"].max()
z = rental["inventory_id"].max() # from the df "rental", we know that the inventory id is a much higher number that inventory_id in the df "inventory"
aproximate_number_of_total_films = (y*z)/x

aproximate_number_of_total_films # It seems that the number of films that the company owns is around 1000

1021.563

In [24]:
# Delete the column "store_id".

inventory = inventory.drop("store_id", axis=1)

In [25]:
# Change the name of the column "last_update" by "last_inventory_date".

inventory.rename(columns={"last_update": "last_inventory_date"}, inplace=True)

In [26]:
# Fill the column "last_inventory_date" with default values.

inventory["last_inventory_date"] = "2001-01-01 00:00:00"

inventory.head()

,inventory_id,film_id,last_inventory_date
0,1,1,2001-01-01 00:00:00
1,2,1,2001-01-01 00:00:00
2,3,1,2001-01-01 00:00:00
3,4,1,2001-01-01 00:00:00
4,5,1,2001-01-01 00:00:00


### language

In [27]:
language

,language_id,name,last_update
0,1,English,2006-02-15 05:02:19
1,2,Italian,2006-02-15 05:02:19
2,3,Japanese,2006-02-15 05:02:19
3,4,Mandarin,2006-02-15 05:02:19
4,5,French,2006-02-15 05:02:19
5,6,German,2006-02-15 05:02:19


In [28]:
# Delete the column last_update since it does not contain useful information.

language = language.drop(columns=['last_update'])

language = language.rename(columns={'name': 'language_audio'})

language.head()

,language_id,language_audio
0,1,English
1,2,Italian
2,3,Japanese
3,4,Mandarin
4,5,French


### original_language

In [29]:
# Create a new table named "original_language".

data = {
    'original_language_id': [1, 2, 3, 4, 5, 6, 7],
    'original_language_audio': ['English', 'Italian', 'Japanese', "Mandarin", "French", "German", " Spanish"]
}

original_language = pd.DataFrame(data)

In [30]:
original_language

,original_language_id,original_language_audio
0,1,English
1,2,Italian
2,3,Japanese
3,4,Mandarin
4,5,French
5,6,German
6,7,Spanish


### subtitles

In [31]:
# Create a new table named "subtitles".

data = {
    'subtitles_id': [1, 2, 3, 4, 5, 6, 7],
    'subtitles_language': ['English', 'Italian', 'Japanese', "Mandarin", "French", "German", " Spanish"]
}

subtitles = pd.DataFrame(data)

subtitles

,subtitles_id,subtitles_language
0,1,English
1,2,Italian
2,3,Japanese
3,4,Mandarin
4,5,French
5,6,German
6,7,Spanish


### rental

In [32]:
rental.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [33]:
len(rental['inventory_id'].unique())

1000

In [34]:
rental["inventory_id"].max()

4581

In [35]:
# Delete the columns "last_update" and "staff_id".

rental = rental.drop(columns = ["last_update"])

rental = rental.drop (columns = ["staff_id"])

rental.head()

,rental_id,rental_date,inventory_id,customer_id,return_date
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21


In [36]:
# Fill the columns "inventory_id" and "customer_id" with default values.

rental["inventory_id"] = 0

rental["customer_id"] = 0
rental.head()


,rental_id,rental_date,inventory_id,customer_id,return_date
0,1,2005-05-24 22:53:30,0,0,2005-05-26 22:04:30
1,2,2005-05-24 22:54:33,0,0,2005-05-28 19:40:33
2,3,2005-05-24 23:03:39,0,0,2005-06-01 22:12:39
3,4,2005-05-24 23:04:41,0,0,2005-06-03 01:43:41
4,5,2005-05-24 23:05:21,0,0,2005-06-02 04:33:21


In [37]:
# Insert defalut values in "rental_date" and "return_date".

rental["return_date"] = "2001-01-01 00:00:00"

rental["rental_date"] = "2001-01-01 00:00:00"

rental.head()

,rental_id,rental_date,inventory_id,customer_id,return_date
0,1,2001-01-01 00:00:00,0,0,2001-01-01 00:00:00
1,2,2001-01-01 00:00:00,0,0,2001-01-01 00:00:00
2,3,2001-01-01 00:00:00,0,0,2001-01-01 00:00:00
3,4,2001-01-01 00:00:00,0,0,2001-01-01 00:00:00
4,5,2001-01-01 00:00:00,0,0,2001-01-01 00:00:00


In [38]:
# Add 3 new columns "customer_name", "telephone" and "email".

In [39]:
rental["customer_name"] = "Unknown"
rental["telephone"] = "+34000000000"
rental["email"] = "unknown@unknown.com"

rental.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,customer_name,telephone,email
0,1,2001-01-01 00:00:00,0,0,2001-01-01 00:00:00,Unknown,+34000000000,unknown@unknown.com
1,2,2001-01-01 00:00:00,0,0,2001-01-01 00:00:00,Unknown,+34000000000,unknown@unknown.com
2,3,2001-01-01 00:00:00,0,0,2001-01-01 00:00:00,Unknown,+34000000000,unknown@unknown.com
3,4,2001-01-01 00:00:00,0,0,2001-01-01 00:00:00,Unknown,+34000000000,unknown@unknown.com
4,5,2001-01-01 00:00:00,0,0,2001-01-01 00:00:00,Unknown,+34000000000,unknown@unknown.com


### old HDD

In [40]:
old_HDD.head()

,first_name,last_name,title,release_year,category_id
0,PENELOPE,GUINESS,ACADEMY DINOSAUR,2006,6
1,PENELOPE,GUINESS,ANACONDA CONFESSIONS,2006,2
2,PENELOPE,GUINESS,ANGELS LIFE,2006,13
3,PENELOPE,GUINESS,BULWORTH COMMANDMENTS,2006,10
4,PENELOPE,GUINESS,CHEAPER CLYDE,2006,14


In [41]:
# Crete a new column with the complete name of the actor/actress in lowercase.

old_HDD = old_HDD.assign(actor_actress_name = (old_HDD['first_name'] + ' ' + old_HDD['last_name']).str.lower())
old_HDD.head()

,first_name,last_name,title,release_year,category_id,actor_actress_name
0,PENELOPE,GUINESS,ACADEMY DINOSAUR,2006,6,penelope guiness
1,PENELOPE,GUINESS,ANACONDA CONFESSIONS,2006,2,penelope guiness
2,PENELOPE,GUINESS,ANGELS LIFE,2006,13,penelope guiness
3,PENELOPE,GUINESS,BULWORTH COMMANDMENTS,2006,10,penelope guiness
4,PENELOPE,GUINESS,CHEAPER CLYDE,2006,14,penelope guiness


In [42]:
# Check if there are duplicated rows

duplicates = old_HDD.duplicated()
duplicates.value_counts()

False    1000
dtype: int64

In [43]:
# Delete columns: "first_name" and "last_name".

old_HDD = old_HDD.drop(columns=['first_name', 'last_name'])
old_HDD.head()

,title,release_year,category_id,actor_actress_name
0,ACADEMY DINOSAUR,2006,6,penelope guiness
1,ANACONDA CONFESSIONS,2006,2,penelope guiness
2,ANGELS LIFE,2006,13,penelope guiness
3,BULWORTH COMMANDMENTS,2006,10,penelope guiness
4,CHEAPER CLYDE,2006,14,penelope guiness


In [44]:
# Change the name of the table "old_HDD" by "actor_per_film".

actor_per_film = old_HDD.copy()

actor_per_film["actor_per_film_id"] = range(len(actor_per_film))

actor_per_film

,title,release_year,category_id,actor_actress_name,actor_per_film_id
0,ACADEMY DINOSAUR,2006,6,penelope guiness,0
1,ANACONDA CONFESSIONS,2006,2,penelope guiness,1
2,ANGELS LIFE,2006,13,penelope guiness,2
3,BULWORTH COMMANDMENTS,2006,10,penelope guiness,3
4,CHEAPER CLYDE,2006,14,penelope guiness,4
5,COLOR PHILADELPHIA,2006,4,penelope guiness,5
6,ELEPHANT TROJAN,2006,11,penelope guiness,6
7,GLEAMING JAWBREAKER,2006,15,penelope guiness,7
8,HUMAN GRAFFITI,2006,10,penelope guiness,8
9,KING EVOLUTION,2006,8,penelope guiness,9


# Export the tables as .csv documents

In [45]:
#inventory.to_csv('/Users/david/Desktop/IronHack/Projects/SQL_project/data/cleaned/inventory.csv', index=False)

In [46]:
#film.to_csv("/Users/david/Desktop/IronHack/Projects/SQL_project/data/cleaned/film.csv",index=False)

In [47]:
#language.to_csv("/Users/david/Desktop/IronHack/Projects/SQL_project/data/cleaned/language.csv",index=False)

In [48]:
#original_language.to_csv("/Users/david/Desktop/IronHack/Projects/SQL_project/data/cleaned/original_language.csv",index=False)

In [49]:
#subtitles.to_csv("/Users/david/Desktop/IronHack/Projects/SQL_project/data/cleaned/subtitles.csv",index=False)

In [50]:
#actor.to_csv("/Users/david/Desktop/IronHack/Projects/SQL_project/data/cleaned/actor.csv",index=False)

In [51]:
#category.to_csv("/Users/david/Desktop/IronHack/Projects/SQL_project/data/cleaned/category.csv",index=False)

In [52]:
# actor_per_film.to_csv("/Users/david/Desktop/IronHack/Projects/SQL_project/data/cleaned/actor_per_film.csv",index=False)

In [53]:
#rental.to_csv("/Users/david/Desktop/IronHack/Projects/SQL_project/data/cleaned/rental.csv",index=False)